## 코랩에서 Selenium 실행하기 (둘 중 하나만 실행)

In [ ]:
!pip install kora -q
from kora.selenium import wd

## 로컬에서 Selenium 실행하기 (둘 중 하나만 실행)

In [ ]:
from selenium import webdriver

In [ ]:
wd = webdriver.Chrome("./programming/PycharmProjects/chromedriver")

## Selenium, pandas, os, time 모듈 불러오기

In [ ]:
import os, time
import pandas as pd

## 드라이버로 웹페이지 열고 "더보기" 두 번 누르기

In [ ]:
CORONABOARD_URL = "https://coronaboard.kr"
wd.get(CORONABOARD_URL)

In [ ]:
time.sleep(2)
load_more_button = wd.find_element_by_id("show-more")
for _ in range(2):
  time.sleep(4)
  load_more_button.click()

## 국가별 정보 가져오기 - (Row, 행)

In [ ]:
start_time = time.time()    # 타이머 시작
all_data = []               # 전체 데이터

# xpath를 통해 국가들 가져오기
rows = wd.find_elements_by_xpath(f"""//*[@id="country-table"]/div/div/table/tbody/tr""")

# 국가별로 데이터 처리하기
for row in rows:
  # 여러개의 td 요소 찾기
  columns = row.find_elements_by_tag_name("td")
  confirmed = int("".join(columns[2].text.split("\n")[0].split(",")))    # 확진자수
  death_toll = int("".join(columns[3].text.split("\n")[0].split(",")))   # 사망자수
  
  # 완치자수 정보가 없는 경우, continue를 통해 현재 국가 건너뛰기
  if columns[4].text != "N/A":
    recovered = int("".join(columns[4].text.split("\n")[0].split(",")))
  else:
    continue
  
  # 발생률 정보가 없는 경우, continue를 통해 현재 국가 건너뛰기
  if columns[7].text != "-":
    confirmed_rate = int("".join(columns[7].text.split(",")))
    population = int(confirmed / confirmed_rate * 1000000)      # 인구 = 확진자수 * 발생률 * 1,000,000
  else:
    continue

  death_per_pop = round(int(death_toll) / population * 100, 5)  # % 단위, 소숫점 다섯째짜리까지 가져오기

  # 국가별로 all_data에 딕셔너리로 넣어주기
  all_data.append({
      "국가": columns[1].text,
      "확진자": confirmed,
      "사망자": death_toll,
      "완치": recovered,
      "발생률": confirmed_rate,
      "인구": population,
      "인구 대 사망자수": death_per_pop
  })

end_time = time.time()    ## 타이머 종료
print(f"소요시간: {round(end_time-start_time)}초")

소요시간: 27초


In [ ]:
covid_data = pd.DataFrame(all_data)
print(covid_data)

           국가       확진자     사망자        완치     발생률          인구  인구 대 사망자수
0        미국🇺🇸  34419838  617463  28767507  103987   331001355    0.18654
1        인도🇮🇳  29977861  389302  28926038   21723  1380005570    0.02821
2       브라질🇧🇷  17969806  502817  16288392   84540   212559806    0.23655
3       프랑스🇫🇷   5757798  110778   5566394   88210    65273755    0.16971
4        터키🇹🇷   5375593   49236   5237731   63738    84338903    0.05838
..        ...       ...     ...       ...     ...         ...        ...
211   마셜 제도🇲🇭         4       0         4      68       58823    0.00000
212    바누아투🇻🇺         4       1         3      13      307692    0.00033
213     사모아🇼🇸         3       0         3      15      200000    0.00000
214  세인트헬레나🇸🇭         2       0         2     329        6079    0.00000
215  미크로네시아🇫🇲         1       0         1       9      111111    0.00000

[216 rows x 7 columns]


In [ ]:
# 인구수로 정렬하고, 28개 국가까지 자르기 (인덱스 초기화)
covid_data = covid_data.sort_values(by = ["인구"], ascending=False).reset_index(drop=True).iloc[:28,:]
# 인구 대 사망자수로 정렬하기 (인덱스 초기화)
covid_data = covid_data.sort_values(by = ["인구 대 사망자수"], ascending=False).reset_index(drop=True)
# csv로 내보내기
covid_data.to_csv("covid_data.csv")
print("파일이 저장된 경로:", os.getcwd())

파일이 저장된 경로: /content
